<a href="https://colab.research.google.com/github/abhilashhn1993/Sentiment_Analysis_of_Tweets/blob/master/AnchoredCorexApproach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk; 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install spacy
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 552kB 44.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=52706dd1de63d10bb663c018cadee55aead3b287224be3c41c2453f55c4aadc6
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=7f69c86ea4c71dde9c4494c4ea642097c39a90fd913aa11242f144ea23666347
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [0]:
#Adding Extra Stopwords to be removed from the dataset
stop_words.extend(['http', 'https', 'twitter', 'www', 'instagram', 'zsgdbw', 'tmblr', 'co'])

In [0]:
stop_words.extend(['make','really','see','go','would','even','get','com','still','do','know','actually','much','let','s','have','name','photo'])

In [0]:
stop_words.extend(['kit','folk','straw',''])

In [10]:
from google.colab import files
uploaded = files.upload()

Saving post_lammetized.csv to post_lammetized.csv
Saving pre_lammetized.csv to pre_lammetized.csv


In [0]:
import io
df_pre = pd.read_csv(io.BytesIO(uploaded['pre_lammetized.csv']))
df_post = pd.read_csv(io.BytesIO(uploaded['post_lammetized.csv']))

In [12]:
df_pre.head()

,Tweets
0,damn read lucky think hell lucky
1,good luck send support ray
2,stan little boy think seb old problem concept ...
3,delete folder
4,wear shoe cause lazy put kid tip tap tip tap


In [0]:
df = df_pre
df2 = df_post

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

In [0]:
vectorizer = vectorizer.fit(df_pre['Tweets'])
tfidf = vectorizer.transform(df_pre['Tweets'])
vocab = vectorizer.get_feature_names()

In [16]:
print(len(vocab))

1838


In [17]:
!pip install corextopic

  Created wheel for corextopic: filename=corextopic-1.0.5-cp36-none-any.whl size=22374 sha256=aff40665d1f4b551271cdaf17f92e5abb8a4deae03cde881265e649b52bb5e82
  Stored in directory: /root/.cache/pip/wheels/a8/1c/04/7f75e4433b37b770736e7cd017775fb997b9f3222b120da15f
Successfully built corextopic


In [18]:
from corextopic import corextopic as ct

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
anchors = []
pre_model = ct.Corex(n_hidden=4, seed=42)
pre_model = pre_model.fit(
    tfidf,
    words=vocab
)

In [20]:
for i, topic_ngrams in enumerate(pre_model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: disabled, folk, disabled people, game, issue, time, forward, accessibility, straw, look forward
Topic #2: people, feel, say, friend, never, bad, want, think, life, good friend
Topic #3: sleep, home, tomorrow, birthday, morning, eat, happy birthday, happy, night, weight
Topic #4: chroniclife, need, pain, may, help, o¾œ, new, hand, sure, surgery


In [30]:
#Giving Anchors
anchors1 = [
           ["trauma","accident","assault","disabled"], #CAUSES
           ["sad","alone","bad","hurt","feel","good","pain"], #FEELINGS AND EMOTIONS
           ["anxiety", "nightmare","sleep","nervous", "tired"], #SYMPTOMS BOTH PHYSICAL & MENTAL
           ["depressed","fuck","shit","lost","mental","die"] #BEHAVIOUR & STATE OF MIND
]
pre_model2 = ct.Corex(n_hidden=4, seed=42)
pre_model2 = pre_model2.fit(
    tfidf,
    words=vocab,
    anchors=anchors1,
    anchor_strength=5
)

In [22]:
for i, topic_ngrams in enumerate(pre_model2.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: disabled, people, assault, folk, trauma, disabled people, issue, accessibility, think, forward
Topic #2: feel, good, bad, pain, sad, hurt, alone, good friend, feel bad, look good
Topic #3: sleep, tired, anxiety, nervous, nightmare, fall asleep, chroniclife raredisease, lose weight, raredisease, eat disorder
Topic #4: shit, fuck, die, mental, depressed, illness, mental health, mental illness, social postshare, postshare


In [0]:
topic_df = pd.DataFrame(
    pre_model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(4)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)

In [25]:
df.sample(10, random_state=123)

,Tweets,topic_1,topic_2,topic_3,topic_4
14913,want doctor rather pain fault,0.0,0.0,0.0,1.0
5286,move elsewhere ill switch regular night schedu...,0.0,0.0,1.0,1.0
8558,hard deal life sometimes spend majority time h...,0.0,1.0,0.0,0.0
3205,sad cause always old,0.0,1.0,0.0,0.0
286,want look,0.0,0.0,0.0,0.0
5902,need surround positive vibe,0.0,0.0,0.0,0.0
2522,disrespect rocky,0.0,0.0,0.0,0.0
5196,comment ago well block could respond super lat...,0.0,0.0,0.0,0.0
18734,chemical burn,0.0,0.0,0.0,0.0
22207,big mood,0.0,0.0,0.0,0.0


In [0]:
from google.colab import files
df.to_csv('pre_diagnosis_topicModeled.csv') 
files.download('pre_diagnosis_topicModeled.csv')

**For Post Tweets**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer2 = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

In [0]:
vectorizer2 = vectorizer2.fit(df_post['Tweets'])
tfidf_post = vectorizer2.transform(df_post['Tweets'])
vocab_post = vectorizer2.get_feature_names()

In [0]:
anchors_post = []
post_model = ct.Corex(n_hidden=7, seed=42)
post_model = post_model.fit(
    tfidf_post,
    words=vocab_post
)

In [38]:
for i, topic_ngrams in enumerate(post_model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: people, game, disabled, accessibility, think, folk, many, community, hard, never
Topic #2: feel, take, help, long, forward, disorder, care, look forward, health, could
Topic #3: next, week, hour, tonight, morning, night, sleep, tomorrow, sheet, home
Topic #4: time, shit, ass, tell, say, first, want, first time, fuck, give
Topic #5: good, friend, good friend, birthday, happy, happy birthday, music, look good, video, idea
Topic #6: support, message, struggle, woman, ask, question, text, character, film, comment
Topic #7: girl, fall, sweet, boy, cold, cute, lol, baby, hot, scream


In [0]:
#Giving Anchors
anchors2 = [
           ["no sleep","tired","nightmare","anxiety","chronic"], #SYMPTOMS
           ["sad","lonely","bad","hurt","feel","anger","pain"], #FEELINGS AND EMOTIONS
           ["drug","pill","antidepressant"], #MEDICATIONS AND DRUGS
           ["therapy","community","doctor","support","session"], #THERAPY AND SUPPORT
           ["help","need","want","love","share","talk","tell"], #CALL FOR HELP
           ["positive","motivate","glad","accept","self","better"] #SELF-IMPROVEMENT  
]

In [43]:
post_model2 = ct.Corex(n_hidden=6, seed=42)
post_model2 = post_model2.fit(
    tfidf,
    words=vocab,
    anchors=anchors2,
    anchor_strength=3
)

In [44]:
for i, topic_ngrams in enumerate(post_model2.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: tired, anxiety, people, chronic, mental, user, travel, issue, think, mile
Topic #2: feel, bad, pain, sad, hurt, lonely, feel bad, anger, feel good, feel well
Topic #3: drug, say, friend, pill, never, good friend, cause, many, could, good
Topic #4: support, disabled, doctor, community, therapy, folk, disabled people, accessibility, straw, look forward
Topic #5: love, want, need, tell, help, talk, share, thank love, need new, people tell
Topic #6: better, self, time, glad, chroniclife, take, accept, home, positive, work
